In [ ]:
import argparse
import sys
import os
from skimage import io

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, utils
import matplotlib.patches as mpatches

from tqdm import tqdm

# from scheduler import CycleScheduler
from pt_utils import  Embeddings, Trainer, VQVAE, data_sampler
from torch.utils import data
from torch import distributed as dist

from umap import UMAP
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from collections import Counter
from skimage import transform, metrics
import skimage
import pandas as pd
import numpy as np
import glob

from torchsummary import summary

# from tqdm.notebook import trange, tqdm
from tqdm import trange, tqdm
import torchvision
from PIL import Image

import numpy as np
from sklearn.decomposition import PCA

In [ ]:
seed = 51
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
device = "cuda"
model =    VQVAE(in_channel=3,
           channel=128,
           n_res_block=6,
           n_res_channel=32,
           embed_dim=1,
           n_embed=8192,
           decay=0.99).to(device)


In [ ]:
# summary(model, input_size=(3, 512, 512))

In [ ]:
# dataset_path = '../data/dataset_512/'
# dataset_path = '../datasets/bc_right_sub_left_minmax_4x_360'
dataset_path = '../datasets/bc_left_4x_360'

resize_shape = (512, 512)
# resize_shape = (1024, 1024)

n_gpu = 1
batch_size = 8
val_split = 0.15

transform = transforms.Compose(
    [
        # transforms.Resize(resize_shape),
        transforms.CenterCrop(resize_shape),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ]
)

dataset = datasets.ImageFolder(dataset_path, transform=transform)

train_dataset_len = int(len(dataset) * (1 - val_split))
test_dataset_len = len(dataset) - train_dataset_len

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_dataset_len, test_dataset_len],
                                                            generator=torch.Generator().manual_seed(seed))

train_sampler = data_sampler(train_dataset, shuffle=True, distributed=False)
test_sampler = data_sampler(test_dataset, shuffle=True, distributed=False)

train_loader = DataLoader(
    train_dataset, batch_size=batch_size // n_gpu, sampler=train_sampler, num_workers=2
)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size // n_gpu, sampler=test_sampler, num_workers=2
)

In [ ]:
# model_file = 'data/logs/emb_dim_1_n_embed_8192_bc_left_sub_right_minmax_4x_360/vqvae_001_train_0.04914_test_0.04206.pt'
model_file = 'data/logs/emb_dim_1_n_embed_8192_bc_right_sub_left_minmax_4x_360/vqvae_003_train_0.04287_test_0.04129.pt'

model.load_state_dict(torch.load(model_file, map_location=torch.device('cuda')))

In [ ]:
epochs = 100
lr = 1e-4

latent_loss_weight = 0.25
sample_size = 25

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-7, amsgrad=True)
# optimizer = optim.RMSprop(model.parameters(), lr=lr,weight_decay=1e-6,centered=True)

# sample_path = '../data/logs/vq-vae-2/4x/samples'
# model_path = '../data/logs/vq-vae-2/4x/weights'

# model_path = 'data/logs/emb_dim_1_n_embed_8192_bc_right_sub_left_minmax_4x_360'
model_path = 'data/logs/emb_dim_1_n_embed_8192_bc_left_4x_360'

Trainer.train(model=model, optimizer=optimizer, train_loader=train_loader, test_loader=test_loader,
              model_path=model_path, epochs=epochs)


In [ ]:
sample_path = '../data/logs/vq-vae-2/samples'

with torch.no_grad():
    for j, (img, label) in enumerate(test_loader):
        img = img.to(device)
        out, latent_loss = model(img)

        sample_size = 25
        sample = img[:sample_size]

        utils.save_image(
            torch.cat([sample, out], 0),
            f"{sample_path}/{j}_vq_vae_2_test.png",
            nrow=sample_size,
            normalize=True,
            range=(-1, 1),
        )

## Визуализация

In [ ]:
model_file = 'data/logs/emb_dim_1_n_embed_8192_bc_left_4x_360/vqvae_004_train_0.02043_test_0.02022.pt'
# model_file = 'data/logs/emb_dim_1_n_embed_8192_bc_left_sub_right_minmax_4x_360/vqvae_002_train_0.04045_test_0.03895.pt'
# model_file = 'data/logs/emb_dim_1_n_embed_8192_bc_right_sub_left_minmax_4x_360/vqvae_003_train_0.04287_test_0.04129.pt'
# model_file = '../data/logs/vq-vae-2/4x_emb_dim_32/weights/vqvae_011_train_0.00375_test_0.00375.pt'
# model_file = '../data/logs/vq-vae-2/4x/weights/vqvae_013_train_0.0038_test_0.0038.pt'
# model_file = '../data/logs/vq-vae-2/vqvae_100_train_0.00409_test_0.00401_512.pt'

device = "cuda"
model = VQVAE(in_channel=3,
       channel=128,
       n_res_block=6,
       n_res_channel=32,
       embed_dim=1,
       n_embed=8192,
       decay=0.99).to(device)

model.load_state_dict(torch.load(model_file, map_location=torch.device('cuda')))

In [ ]:
# dataset_path = '../datasets/bc_right_sub_left_minmax_4x_360'
# dataset_path = '../datasets/bc_left_sub_right_0.5_4x_360'
dataset_path = '../datasets/bc_right_sub_left_0.5_4x_360'
# dataset_path = '../datasets/bc_left_sub_right_minmax_4x_360'
# dataset_path = '../datasets/bc_left_4x_360'

resize_shape = (512, 512)

transform = transforms.Compose(
    [
        # transforms.Resize(resize_shape),
        transforms.CenterCrop(resize_shape),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ]
)

dataset = datasets.ImageFolder(dataset_path, transform=transform)

In [ ]:
embs_t[0]

In [ ]:
embs_b[0]

In [ ]:
embs_t, embs_b = Embeddings.get_vqvae2_embs(model, dataset)

In [ ]:
np.save('embs_t_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left.npy', embs_t)
np.save('embs_b_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left.npy', embs_b)

In [ ]:
pca = PCA(n_components=2,svd_solver='full')
pca.fit(embs_t)
umaped_vct_2d_t=pca.transform(embs_t)

pca = PCA(n_components=2,svd_solver='full')
pca.fit(embs_b)
umaped_vct_2d_b=pca.transform(embs_b)

In [ ]:
embs_t=np.load('data/embs/embs_t_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left.npy')
embs_b=np.load('data/embs/embs_b_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left.npy')

In [ ]:
np.random.seed(51)
umap_2d = UMAP(random_state=seed,n_components=2)

embs_b=np.array(embs_b)
embs_t=np.array(embs_t)

umaped_vct_2d_t = umap_2d.fit_transform(embs_t)
umaped_vct_2d_b = umap_2d.fit_transform(embs_b)

In [ ]:
embs_t

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
    

colors = ['b', 'g', 'y', 'm', 'c']
markers = ['8', 'v', 's', 'd', '*', ]

scatter_xy = embs_scatter.reshape(r_shape)

for i in range(len(scatter_xy)):
    ax.scatter(scatter_xy[i, :, 0], scatter_xy[i, :, 1], color=colors[i], s=dot_size,
               marker=markers[i])

ax.legend(legend, fontsize=fontsize)
# if save and name:
#     plt.savefig(f'{name}.png', bbox_inches = 'tight')
plt.show()

In [ ]:
seed=51


types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}

classes = dataset.__dict__['classes']

legend = [cls + '\n' + types_dict[cls] for cls in classes]

# name_1 = f'embs_space_seed={seed}_t_512_4x_360_minmax_emb_dim_1_n_embed_8192_right_sub_left_unique_model'
name_1 = f'embs_space_seed={seed}_t_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left' 
# name_2 = f'embs_space_seed={seed}_b_512_4x_360_minmax_emb_dim_1_n_embed_8192_right_sub_left_unique_model'
name_2 = f'embs_space_seed={seed}_b_512_4x_360_emb_dim_1_n_embed_8192_right_sub_left_0.5_trunc_model_left'
r_shape = (len(classes), -1, 2)

# save=True
save=False

Embeddings.plot_2d_scatter_embs(embs_scatter=umaped_vct_2d_t, legend=legend, r_shape=r_shape, save=save, name=name_1, N=10, M=10)
Embeddings.plot_2d_scatter_embs(embs_scatter=umaped_vct_2d_b, legend=legend, r_shape=r_shape, save=save, name=name_2, N=10, M=10)

# Ошибки

In [ ]:
mse_losses = []
ssim_losses = []

visual_loader = DataLoader(
    dataset, batch_size=1, num_workers=2
)

for i, (img, label) in enumerate(visual_loader):
    model.zero_grad()
    img = img.to(device)

    out, latent_loss = model(img)
    predicted_image = np.transpose(out.cpu().detach().numpy()[0])
    original_image = np.transpose(img.cpu().detach().numpy()[0])

    mse_losses.append(metrics.mean_squared_error(original_image, predicted_image))
    ssim_losses.append(metrics.structural_similarity(original_image, predicted_image, multichannel=True))

mse_losses = np.array(mse_losses)
ssim_losses = np.array(ssim_losses)

In [ ]:
df_mse = pd.DataFrame()
df_ssim = pd.DataFrame()


In [ ]:
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 15))

names = ['Ultra_Co8\nсредние зерна', 'Ultra_Co11\nмелкие зерна', 'Ultra_Co6_2\nмелкие зерна',
         'Ultra_Co15\nкрупные зерна', 'Ultra_Co25\nсредне-мелкие зерна']

colors = ['b', 'g', 'y', 'm', 'c']
markers = ['8', 'v', 's', 'd', '*', ]

f_vects = []

cnt = Counter(np.array(dataset.samples)[:, 1])

for i in cnt.keys():
    emb_number = cnt[i]
    key = int(i)

    start = 0
    for val in range(int(i)):
        start += cnt[str(val)]

    end = start + emb_number

    mse_y = mse_losses[start:end]
    ssim_y = ssim_losses[start:end]

    df_mse[names[key]] = pd.Series(mse_y)
    df_ssim[names[key]] = pd.Series(ssim_y)

plt.rcParams['font.size'] = '20'

# plt.ylabel('MSE loss', fontsize=20)
name = 'vq-vae-2'
# plt.savefig(f'mse_ssim_losses_{name}_512.png')

# plt.show()

In [ ]:
plt.rcParams['font.size'] = '15'
name = 'vq-vae-2'

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 12))
# plt.figure(figsize=(10,10))
df_ssim.boxplot(ax=ax2)
df_mse.boxplot(ax=ax1)

ax1.set_ylabel('MSE loss', fontsize=20)
ax2.set_ylabel('SSIM similarity', fontsize=20)

plt.savefig(f'mse_ssim_losses_{name}_512_boxplot.png')

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(30, 15))

names = ['Ultra_Co8\nсредние зерна', 'Ultra_Co11\nмелкие зерна', 'Ultra_Co6_2\nмелкие зерна',
         'Ultra_Co15\nкрупные зерна', 'Ultra_Co25\nсредне-мелкие зерна']

colors = ['b', 'g', 'y', 'm', 'c']
markers = ['8', 'v', 's', 'd', '*', ]

f_vects = []

cnt = Counter(np.array(dataset.samples)[:, 1])

for i in cnt.keys():
    emb_number = cnt[i]
    key = int(i)

    start = 0
    for val in range(int(i)):
        start += cnt[str(val)]

    end = start + emb_number

    x = np.arange(mse_losses.shape[0])[start:end]
    mse_y = mse_losses[start:end]
    ssim_y = ssim_losses[start:end]

    # ax1.plot(x, mse_y)
    ax1.scatter(x, mse_y)

    # ax2.plot(x, ssim_y)
    ax2.scatter(x, ssim_y)

plt.rcParams['font.size'] = '20'

ax1.legend(names, fontsize=15)
ax1.set_title('MSE image losses', fontsize=20)
ax1.set_xlabel('image number', fontsize=20)
ax1.set_ylabel('MSE loss', fontsize=20)

ax2.legend(names, fontsize=15)
ax2.set_title('Structural image similarity', fontsize=20)
ax2.set_xlabel('image number', fontsize=20)
ax2.set_ylabel('SSIM similarity', fontsize=20)

name = 'vq-vae-2'
plt.savefig(f'mse_ssim_losses_{name}_512.png')

plt.show()

In [ ]:
def get_emb_distr(f_vects, projection_axis, step, eps):
    """
    :param step:
    :param f_vects: ndarray shape (classes_num,N,2)
    :param projection_axis:
    :param eps:
    :return:
    """

    axis_start = f_vects[0, :, projection_axis].min()
    axis_end = f_vects[0, :, projection_axis].max()

    for vects in f_vects:
        min_val = vects[:, projection_axis].min()
        max_val = vects[:, projection_axis].max()

        if min_val < axis_start:
            axis_start = min_val

        if max_val > axis_end:
            axis_end = max_val

    axis_step = step * (axis_end - axis_start)
    axis_eps = eps * (axis_end - axis_start)
    axis_vals = np.arange(axis_start, axis_end, axis_step)

    points_distr = []

    for i, vects in enumerate(f_vects):
        points_distr.append([])
        for axis_val in axis_vals:
            counter = 0
            for point in vects[:, projection_axis]:
                if abs(point - axis_val) <= axis_eps:
                    counter += 1
            points_distr[i].append(counter)

    for i, distr in enumerate(points_distr):
        points_distr[i] = np.array(points_distr[i]) / sum(distr)

    return axis_vals, np.array(points_distr), axis_step, axis_eps

In [ ]:
names = ['Ultra_Co8\nсредние зерна', 'Ultra_Co11\nмелкие зерна', 'Ultra_Co6_2\nмелкие зерна',
         'Ultra_Co15\nкрупные зерна', 'Ultra_Co25\nсредне-мелкие зерна']

colors = ['b', 'g', 'y', 'm', 'c']
markers = ['8', 'v', 's', 'd', '*', ]

f, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 20))

title = 'VQ-VAE-2 embs projections'

ax1.set_title(f'{title}', fontsize=20)
ax1.set_xlabel('y val', fontsize=20)
ax1.set_ylabel('norm(count)', fontsize=20)

ax2.set_title(f'{title}', fontsize=20)
ax2.set_xlabel('x val', fontsize=20)
ax2.set_ylabel('norm(count)', fontsize=20)

step = 0.05
size = 4
# eps=step/2
eps = 0.2
projection_axis = 1

f_vects = umaped_vct_2d_b.reshape((5, 360, 2))

x_axis_vals, x_points_distr, x_step, x_eps = get_emb_distr(f_vects, projection_axis=0, step=step, eps=eps)
y_axis_vals, y_points_distr, y_step, y_eps = get_emb_distr(f_vects, projection_axis=1, step=step, eps=eps)

for i in range(len(x_points_distr)):
    ax1.plot(y_axis_vals, y_points_distr[i], color=colors[i])
    ax2.plot(x_axis_vals, x_points_distr[i], color=colors[i])

for i in range(len(x_points_distr)):
    # y projection
    ax1.scatter(y_axis_vals, y_points_distr[i], edgecolor=colors[i], s=size, marker=markers[i])

    # x prjection
    ax2.scatter(x_axis_vals, x_points_distr[i], edgecolor=colors[i], s=size, marker=markers[i])

ax1.legend(names + [
    f'шаг={round(y_step, 2)}, доля от длины оси={int(step * 100)}%\neps={round(y_eps, 2)}, доля от длины оси={int(eps * 100)}%'],
           fontsize=15)
ax2.legend(names + [
    f'шаг={round(x_step, 2)}, доля от длины оси={int(step * 100)}%\neps={round(x_eps, 2)}, доля от длины оси={int(eps * 100)}%'],
           fontsize=15)

plt.savefig(f'embs_projection_vq_vae_2_step={step}_eps={eps}.png')
plt.show()